## Comparison for LogSumExp

### N=100

The LogSumExp-problem with linear consraints:
	$$\log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2 \rightarrow \min_{\textbf{x}\in \mathbb{R}^N}$$
	$$\text{s.t.} (\textbf{b}_j, \textbf{x}) - c_j <= 0, j = 1\dots M$$
	
This problem can be converted into saddle point problem:
	$$\min_{\textbf{x}} \max_{\textbf{y}\in\mathbb{R}^M_+} S(\textbf{x},\textbf{y}),$$
where $S(\textbf{x}, \textbf{y}) = r(\textbf{x})+F(\textbf{x}, \textbf{y}) - h(\textbf{y})$
	$$r(\textbf{x}) = \log(1 + \sum\limits_{k=1}^n \exp(\alpha x_k)) + \frac{\beta}{2} \|\textbf{x}\|_2^2$$
	$$F(\textbf{x}, \textbf{y}) = \sum\limits_{j=1}^M x_j(\textbf{b}_j, \textbf{y}) = (\textbf{x}, B\textbf{y})$$
	$$h(\textbf{y}) = (\textbf{c}, \textbf{y})$$

So N is dimensional of external variable, M - internal. But we will consider the external problem as help problem.

In [1]:
import numpy as np
from TestFunctions import LogSumExp
from TestFunctions.TestFunctions import TestFunction
from Solvers.GradientMethods import FGM_internal
import matplotlib.pyplot as plt
%matplotlib notebook
import comparison
import imp
from comparison import method_comparison, create_methods_dict

In [2]:
N = 100
M = 2
alpha = 1
beta = 0.001
B = np.random.uniform(low = -1, high = 1, size = (M, N))
c = np.random.uniform(low = 0, high = 1, size = (M, ))

In [3]:
x_ = np.zeros((N,))
f_ = LogSumExp.r(alpha, beta).get_value(x_)
gamma = c.max()
y_max = f_/gamma
Q = [[0, y_max] for i in range(M)]
Q = np.array(Q)
size_domain = np.linalg.norm(Q[:,0]-Q[:,1])

In [4]:
LSE = TestFunction(r = LogSumExp.h(c, size_domain = size_domain),
                F = LogSumExp.F_tilde(B, size_domain),
                h = LogSumExp.r(alpha, beta, size_domain),
                solver = FGM_internal, 
                get_start_point = lambda x: (1/beta * -x.dot(B), alpha * np.sqrt(N)/beta))

In [5]:
history = {}
eps = 1e-10
methods = create_methods_dict(LSE, Q[:,1], size_domain, Q, eps, history, time_max = 10)

In [6]:
imp.reload(comparison)
comparison.method_comparison(methods)

/home/elias/Desktop/PROJECTS/Optimization-Halving-The-Square/SPP/Solvers/GradientMethods.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  k = min(np.sqrt(4 * L * R/ eps), 2 * np.sqrt(f.L_xx/f.mu_x) * np.log(L * R/ eps))
/home/elias/Desktop/PROJECTS/Optimization-Halving-The-Square/SPP/Solvers/GradientMethods.py:10: RuntimeWarning: divide by zero encountered in double_scalars
  z = min(1/3 * k+ 2.4, 1 + np.sqrt(f.L_xx/f.mu_x))


In [7]:
keys = ['Ellipsoids', 'Dichotomy', 'FGM']

In [10]:
def get_plot(history, f, keys = None):
    if keys is None:
        keys = history.keys()
    for key in keys:
        t0 = history[key][0][1]
        t = [i[1]-t0 for i in history[key]]
        values = [f(i[0]) for i in history[key]]
        plt.plot(t, values)
    plt.legend(keys)
    plt.grid()
    plt.xlabel("Time, s")
    plt.ylabel("Values")

In [ ]:
cond = lambda y, R: LSE.L_xx * R <= eps
f = lambda y: LSE.get_value(y, LSE.get_solution(y, cond)[0])
get_plot(history, f, keys = keys)

In [ ]:
LSE.get_value